In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content && git clone https://github.com/ntua-el21050/bd3lms.git
!ls -l /content/bd3lms

Cloning into 'bd3lms'...
remote: Enumerating objects: 1095, done.
remote: Counting objects: 100% (358/358), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 1095 (delta 284), reused 246 (delta 209), pack-reused 737 (from 2)
Receiving objects: 100% (1095/1095), 7.53 MiB | 8.74 MiB/s, done.
Resolving deltas: 100% (693/693), done.
total 4828
-rw-r--r-- 1 root root  862037 Feb  8 13:22  2503.09573v3.pdf
drwxr-xr-x 9 root root    4096 Feb  8 13:22  configs
-rw-r--r-- 1 root root   33822 Feb  8 13:22  dataloader.py
-rw-r--r-- 1 root root   46696 Feb  8 13:22  diffusion.py
-rw-r--r-- 1 root root   56775 Feb  8 13:22  extension1_edm.ipynb
-rw-r--r-- 1 root root   58511 Feb  8 13:22  extension1_fm.ipynb
-rw-r--r-- 1 root root   58620 Feb  8 13:22  extension1_iddpm.ipynb
-rw-r--r-- 1 root root   58457 Feb  8 13:22  extension1_sigmoid.ipynb
-rw-r--r-- 1 root root   59098 Feb  8 13:22  extension1_sigmoid_val.ipynb
-rw-r--r-- 1 root root   58505 Feb  8 13:22  extension1_simple

In [ ]:
!ls -l /content/drive/MyDrive/DIFF_2

total 4
drwx------ 2 root root 4096 Jan 27 23:24 rework


In [ ]:
!pip install -q \
    torchmetrics==1.6.2 \
    datasets==3.3.2 \
    einops==0.8.1 \
    fsspec==2024.2.0 \
    hydra-core==1.3.2 \
    lightning==2.5.0.post0 \
    omegaconf==2.3.0 \
    packaging==23.2 \
    pandas==2.2.1 \
    rich==13.7.1 \
    scikit-learn==1.5.1 \
    timm==0.9.16 \
    # transformers==4.49.0 \
    matplotlib==3.10.0 \
    wandb
    # numpy.char

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

In [ ]:
from __future__ import annotations

import re
import sys
import subprocess
from pathlib import Path
from typing import Optional
import os

REPO_DIR = Path.cwd()

_ALLOWED_ALGOS = {"ar", "mdlm", "sedd", "bd3lm"}


def _run_command(args: list[str]) -> str:
    env = dict(os.environ)
    env.setdefault("HYDRA_FULL_ERROR", "1")
    proc = subprocess.Popen(
        args,
        cwd=REPO_DIR,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True,
    )

    out_chunks: list[str] = []
    assert proc.stdout is not None
    for line in proc.stdout:
        print(line, end="")  # live logs in Colab/Jupyter
        out_chunks.append(line)

    rc = proc.wait()
    if rc != 0:
        combined = "".join(out_chunks)
        raise RuntimeError(f"Command failed with return code {rc}\nOUTPUT:\n{combined}")

    return "".join(out_chunks)
    return None


def _find_latest_checkpoint(best=False) -> Optional[Path]:
    outputs_dir = REPO_DIR / "outputs"
    if not outputs_dir.exists():
        return None
    if best:
      candidates = list(outputs_dir.rglob("checkpoints/best.ckpt"))
    else:
      candidates = list(outputs_dir.rglob("checkpoints/last.ckpt"))
    if not candidates:
        return None
    return max(candidates, key=lambda p: p.stat().st_mtime)


def train_model(
    *,
    algo="bd3lm",
    block_size=1,
    trainer_max_steps=500,
    sampling_eps_min=1e-3,
    sampling_eps_max=1,
    resume_from_ckpt=False,
    resume_ckpt_path='',
    max_train_samples=500,
    max_valid_samples=100,
    max_test_samples=100,
    nll_diagram=False,
    noise=None,
    noise_mu=None,
    noise_mu1=None,
    noise_w1=None
):
    if algo not in _ALLOWED_ALGOS:
        raise ValueError(f"algo must be one of {_ALLOWED_ALGOS}")


    if not os.path.exists(resume_ckpt_path) and resume_ckpt_path != '':
      print(f"Path {resume_ckpt_path} doeds not exist")
      return None

    else:

      args = [
          sys.executable,
          '-u',
          "bd3lms/main.py",
          "mode=train",
          "model=tiny",
          f"algo={algo}",
          "data=lm1b-wrap",
          "model.length=128",
          "model.attn_backend=sdpa",
          f"block_size={block_size}",
          "trainer.devices=1",
          "loader.global_batch_size=4",
          "loader.batch_size=4",
          "loader.eval_batch_size=4",
          f"trainer.max_steps={trainer_max_steps}",
          f"data.max_train_samples={max_train_samples}",
          f"data.max_valid_samples={max_valid_samples}",
          f"data.max_test_samples={max_test_samples}",
          f"training.nll_diagram={str(nll_diagram)}",
          f"training.sampling_eps_min={sampling_eps_min}",
          f"training.sampling_eps_max={sampling_eps_max}",
          f"checkpointing.resume_from_ckpt={str(resume_from_ckpt)}",
          f"checkpointing.resume_ckpt_path={resume_ckpt_path}",
      ]

      if noise is not None:
          args.append(f"noise={noise}")
      if noise_mu is not None:
          args.append(f"noise.mu={noise_mu}")
      if noise_mu1 is not None:
          args.append(f"noise.mu1={noise_mu1}")
      if noise_w1 is not None:
          args.append(f"noise.w1={noise_w1}")

      _run_command(args)
      latest_ckpt = _find_latest_checkpoint(best=False)
      return str(latest_ckpt) if latest_ckpt else None


def eval_model(
    *,
    algo="bd3lm",
    block_size=1,
    checkpoint_path="/my/path/best.ckpt",
    max_valid_samples=100,
    max_test_samples=100,
    var_min=False,
    sampling_eps_min=0.001,
    sampling_eps_max=1,
    clip_search_widths=[]
):
    if algo not in _ALLOWED_ALGOS:
        raise ValueError(f"algo must be one of {_ALLOWED_ALGOS}")

    args = [
        sys.executable,
        "bd3lms/main.py",
        "mode=ppl_eval",
        "model=tiny",
        f"algo={algo}",
        "data=lm1b-wrap",
        "model.length=128",
        "model.attn_backend=sdpa",
        f"block_size={block_size}",
        "trainer.devices=1",
        "trainer.num_nodes=1",
        "loader.global_batch_size=4",
        "loader.batch_size=4",
        "loader.eval_batch_size=4",
        f"eval.checkpoint_path={checkpoint_path}",
        f"data.max_valid_samples={max_valid_samples}",
        f"data.max_test_samples={max_test_samples}",
        # f"algo.var_min={str(var_min)}",
        # f"training.sampling_eps_min={sampling_eps_min}",
        # f"training.sampling_eps_max={sampling_eps_max}",
        # f"algo.clip_search_widths={str(clip_search_widths)}",
    ]

    output = _run_command(args)

    # Best-effort extraction of perplexity from logs.
    # Common patterns include "val/ppl" or "ppl" in metric logs.
    match_ppl = re.search(r"val/ppl[^\n0-9]*([0-9]+(?:\.[0-9]+)?)", output)
    match_nelbo = re.search(r"valid_var_0.0 - 1[^\n0-9]*([0-9]+(?:\.[0-9]+)?)", output)
    if match_ppl:
        r1 = round(float(match_ppl.group(1)), 2)
        if match_nelbo:
          r2 = round(float(match_nelbo.group(1)), 2)
          return r1, r2
        return (r1, None)
    return None



def print_table(rows):
    if not rows:
        print("No data to display.")
        return

    columns = list(rows[0].keys())

    str_rows = [
        {col: str(row.get(col, "")) for col in columns}
        for row in rows
    ]

    widths = {
        col: max(len(col), max(len(row[col]) for row in str_rows))
        for col in columns
    }

    def print_separator():
        print("+" + "+".join("-" * (widths[col] + 2) for col in columns) + "+")

    def print_row(row):
        print(
            "| " +
            " | ".join(row[col].ljust(widths[col]) for col in columns) +
            " |"
        )

    # Print table
    print_separator()
    print_row({col: col for col in columns})
    print_separator()
    for row in str_rows:
        print_row(row)
    print_separator()


In [ ]:
BASE_DIR = '/content/drive/MyDrive/DIFF_2/rework'
results = []

algos = ['ar', 'mdlm', 'sedd']

for alg in algos:
  model_ckpt = train_model(
      algo=alg,
      trainer_max_steps=500,
      max_train_samples=500,
      max_valid_samples=100,
      max_test_samples=100,
  )

  print(f"Checkpoint for algo: {alg} saved at {model_ckpt}")


In [ ]:
BASE_DIR = '/content/drive/MyDrive/DIFF_2/rework'
results = []

algos = ['ar', 'mdlm', 'sedd']

for alg in algos:
  result = eval_model(
    algo=alg,
    checkpoint_path=BASE_DIR + f'/checkpoints/FINAL_REPR_{alg}/best.ckpt',
    max_valid_samples=100,
    max_test_samples=100,
)

  print(f"Result for algo {alg} is {result}")
  results.append({"model": alg, "block_size_L": '-', "val_ppl": result})

print_table(results)

Seed set to 1
CONFIG
├── mode
│   └── ppl_eval                                                                
├── diffusion
│   └── absorbing_state                                                         
├── seed
│   └── 1                                                                       
├── block_size
│   └── 1                                                                       
├── data
│   └── max_samples: null                                                       
│       max_train_samples: null                                                 
│       max_valid_samples: 100                                                  
│       max_test_samples: 100                                                   
│       train: lm1b                                                             
│       valid: lm1b                                                             
│       tokenizer_name_or_path: bert-base-uncased                               
│       cache_dir: /share/kulesh

In [ ]:
BASE_DIR = '/content/drive/MyDrive/DIFF_2/rework'
results = []

model_ckpt = train_model(
    algo="bd3lm",
    block_size=16,
    trainer_max_steps=500,
    sampling_eps_min = 0.3,
    sampling_eps_max = 0.8,
    resume_from_ckpt=True,
    resume_ckpt_path=BASE_DIR,
    max_train_samples=500,
    max_valid_samples=100,
    max_test_samples=100,
)

print(f"Checkpoint for Block size: 16 saved at {model_ckpt}")


Seed set to 1
CONFIG
├── mode
│   └── train                                                                   
├── diffusion
│   └── absorbing_state                                                         
├── seed
│   └── 1                                                                       
├── block_size
│   └── 16                                                                      
├── data
│   └── max_samples: null                                                       
│       max_train_samples: 500                                                  
│       max_valid_samples: 100                                                  
│       max_test_samples: 100                                                   
│       train: lm1b                                                             
│       valid: lm1b                                                             
│       tokenizer_name_or_path: bert-base-uncased                               
│       cache_dir: /share/kulesh

In [ ]:
!rsync -av \
  /content/outputs/ \
  /content/drive/MyDrive/DIFF_2/rework/synced/

sending incremental file list
./
lm1b/
lm1b/2026.02.01/
lm1b/2026.02.01/091539/
lm1b/2026.02.01/091539/config_tree.txt
lm1b/2026.02.01/091539/main.log
lm1b/2026.02.01/091539/.hydra/
lm1b/2026.02.01/091539/.hydra/config.yaml
lm1b/2026.02.01/091539/.hydra/hydra.yaml
lm1b/2026.02.01/091539/.hydra/overrides.yaml
lm1b/2026.02.01/091539/checkpoints/
lm1b/2026.02.01/091539/checkpoints/16-500.ckpt
lm1b/2026.02.01/091539/checkpoints/best.ckpt
lm1b/2026.02.01/091539/checkpoints/last.ckpt
lm1b/2026.02.01/095950/
lm1b/2026.02.01/095950/config_tree.txt
lm1b/2026.02.01/095950/main.log
lm1b/2026.02.01/095950/.hydra/
lm1b/2026.02.01/095950/.hydra/config.yaml
lm1b/2026.02.01/095950/.hydra/hydra.yaml
lm1b/2026.02.01/095950/.hydra/overrides.yaml
lm1b/2026.02.01/095950/checkpoints/
lm1b/2026.02.01/095950/checkpoints/16-500.ckpt
lm1b/2026.02.01/095950/checkpoints/best.ckpt
lm1b/2026.02.01/095950/checkpoints/last.ckpt
lm1b/2026.02.01/102427/
lm1b/2026.02.01/102427/config_tree.txt
lm1b/2026.02.01/102427/main

In [ ]:

result = eval_model(
    algo="bd3lm",
    block_size=16,
    checkpoint_path=BASE_DIR,
    max_valid_samples=100,
    max_test_samples=100,
)

results.append({"block_size_L": 16, "val_ppl": result[0], "var. NELBO": result[1]})

print_table(results)

Seed set to 1
CONFIG
├── mode
│   └── ppl_eval                                                                
├── diffusion
│   └── absorbing_state                                                         
├── seed
│   └── 1                                                                       
├── block_size
│   └── 16                                                                      
├── data
│   └── max_samples: null                                                       
│       max_train_samples: null                                                 
│       max_valid_samples: 100                                                  
│       max_test_samples: 100                                                   
│       train: lm1b                                                             
│       valid: lm1b                                                             
│       tokenizer_name_or_path: bert-base-uncased                               
│       cache_dir: /share/kulesh